<a href="https://colab.research.google.com/github/gabriel-libardi/PyTorchINOVA-USP/blob/main/Zurique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dataset


In [ ]:
# Baixar e descompactar o dataset
!wget -q "https://docs.toradex.com/private/114105-recyclables_train.zip​"
!wget -q "https://docs.toradex.com/private/114106-recyclables_validation.zip"
#!unzip -q '114105-recyclables_train.zip' -d dataset/

# Definir o caminho do dataset
# data_path = "dataset"

In [ ]:
!unzip -q '114105-recyclables_train.zip' -d dataset/

unzip:  cannot find or open 114105-recyclables_train.zip, 114105-recyclables_train.zip.zip or 114105-recyclables_train.zip.ZIP.


### Importar dependências

In [ ]:
# Carregar a extensão do TensorBoard para o Google Colab
%load_ext tensorboard

# Instalar Pytorch, Torchvision, Tensorboard e utilidades parar ver o progresso do treinamento
!pip install -q torch torchvision torcheval tensorboard matplotlib tqdm tensorflow ipywidgets seaborn

# Módulos, classes e funções que são úteis para inferência e treinamento
import torch, torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_image
from torcheval.metrics import MulticlassF1Score, MulticlassRecall, MulticlassPrecision

# Suporte a TensorBoard no PyTorch
from torch.utils.tensorboard import SummaryWriter

# Utiliades do sistema
from datetime import datetime
import time
import os

# O Tqdm é utilizado para criar barras de progresso
from tqdm.notebook import tqdm


print(torch.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.4 MB/s eta 0:00:00
2.1.0+cu118


### Customização do dataset

In [ ]:
# Declaração da classe herdando de Dataset
class CustomDataset(Dataset):

    # Construtor
    def __init__(self, images_dir, preprocess_function):
        """
        Args:
            images_dir (string): Directory with all the image folders
            preprocess_function (callable): Transform to be applied on a sample
        """

        self.images_dir = images_dir
        self.transform = preprocess_function

        # Ordenamos as classes para termos sempre a mesma ordem
        # Aqui ficam armazenados os nomes de cada classe
        self.classes = sorted(os.listdir(self.images_dir))

        # Caminho para cada imagem
        self.image_paths = []
        # Classe de cada imagem (int)
        self.image_classes = []

        # Procura imagens para todas as classes, em suas respectivas pastas
        for i in range(0, len(self.classes)):
          sample_class = self.classes[i]

          class_dir = os.path.join(self.images_dir, sample_class)
          class_images = os.listdir(class_dir)

          for image in class_images:
            if(not (image.endswith(".jpg") or image.endswith(".JPG") or image.endswith(".png") or image.endswith(".PNG"))):
                continue
            image_path = os.path.join(class_dir, image)
            self.image_paths.append(image_path)
            self.image_classes.append(i)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Checagem por erros na leitura da imagem
        # Isso gera uma nova excessão para que o dataset seja corrigido, não queremos imagens com problemas no dataset
        try:
            sample = read_image(self.image_paths[idx], torchvision.io.ImageReadMode.RGB)
        except:
            print(f"Problem loading image {self.image_paths[idx]}")
            raise Exception()

        # Preprocessamento da imagem
        sample = self.transform(sample)

        # Retorno do par (imagem, classe)
        return sample, self.image_classes[idx]

### Pré-processamento

**OBS.: TESTAR TAMANHOS DE INFERENCE_SIZE DIFERENTES (IMPACTA TEMPO DE EXECUÇÃO) **

In [ ]:
batch_size = 4
inference_size = 50
NormalizationMean = [0.485, 0.456, 0.406]
NormalizationStd = [0.229, 0.224, 0.225]

preprocess_image = transforms.Compose([
    transforms.Resize((inference_size, inference_size), antialias=True),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(mean=NormalizationMean, std=NormalizationStd),
])

### Instanciamento dos datasets

In [ ]:
train_data_path = os.path.join(data_path, "train_crops")
validation_data_path = os.path.join(data_path, "validation_crops")

In [ ]:
train_dataset = CustomDataset(train_data_path, preprocess_image)
validation_dataset = CustomDataset(validation_data_path, preprocess_image)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

print( "Train dataset information:")
print(f" |-> Path:          {train_data_path}")
print(f" |-> Length:        {len(train_dataset)}")
print(f" '-> Loader length: {len(train_dataloader)}")

print( "Validation dataset information:")
print(f" |-> Path:          {validation_data_path}")
print(f" |-> Length:        {len(validation_dataset)}")
print(f" '-> Loader length: {len(validation_dataloader)}")


FileNotFoundError: ignored

### Carregar o modelo

OBS.: TESTAR DIFERENTES MODELOS

In [ ]:
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights=torchvision.models.ResNet18_Weights.DEFAULT)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', weights=torchvision.models.ResNet34_Weights.DEFAULT)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', weights=torchvision.models.ResNet50_Weights.DEFAULT)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', weights=torchvision.models.ResNet101_Weights.DEFAULT)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', weights=torchvision.models.ResNet152_Weights.DEFAULT)
model.eval();

In [ ]:
%tensorboard --logdir runs

### Treinamento por época

In [ ]:
def train_one_epoch(model, dataloader, optimizer, loss_function, epoch_index, device, tensorboard_writer):
    running_loss = 0.0
    last_loss = 0.0

    model.to(device)

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    print(f"Training Epoch {epoch_index}:")

    progress_bar = tqdm(total=len(dataloader))
    for i, data in enumerate(dataloader):

        # Every data instance is an input + label pair
        inputs, labels = data

        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero gradients for every batch
        optimizer.zero_grad()

        # Run Inference on the training data
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_function(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 10 == 9:
            last_loss = running_loss / 10 # loss per batch
            tb_x = (epoch_index*len(train_dataloader)) + i
            tensorboard_writer.add_scalar('Loss/train', last_loss, tb_x)
            tensorboard_writer.flush()
            running_loss = 0.0

        del inputs, labels

        progress_bar.update(1)
    return last_loss

In [ ]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
tensorboard_writer = SummaryWriter(f'runs/model_{timestamp}')

EPOCHS = 1

best_validation_loss = -1

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

device = 'cpu'
if(torch.cuda.is_available()):
    device = 'cuda'

for epoch in range(EPOCHS):
    print(f'EPOCH {epoch}:')

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(model, train_dataloader, optimizer, loss_function, epoch, device, tensorboard_writer)

    running_validation_loss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    model.to(device)

    precision_metric = MulticlassPrecision(average='weighted', num_classes=len(validation_dataset.classes))
    recall_metric = MulticlassRecall(average='weighted', num_classes=len(validation_dataset.classes))
    f1score_metric = MulticlassF1Score(average='weighted', num_classes=len(validation_dataset.classes))

    # Disable gradient computation and reduce memory consumption.
    print(f"Running Validation for Epoch {epoch}")
    with torch.no_grad():
        progress_bar = tqdm(total=len(validation_dataloader))
        for validation_data in validation_dataloader:
            validation_inputs, validation_labels = validation_data
            validation_inputs = validation_inputs.to(device)
            validation_labels = validation_labels.to(device)
            validation_outputs = model(validation_inputs)
            running_validation_loss += loss_function(validation_outputs, validation_labels)

            output_labels = []

            for j in range(0, validation_outputs.size()[0]):
                probabilities = torch.nn.functional.softmax(validation_outputs[j], dim=0)
                prob, det_class = torch.topk(probabilities, 1)
                # If the detected class is outside of bounds, give a wrong result within bounds
                if(det_class >= len(validation_dataset.classes)):
                    det_class = validation_labels[j].item()-1
                    if(det_class < 0):
                        det_class = validation_labels[j].item()+1
                output_labels.append(det_class)

            output_labels = torch.as_tensor(output_labels)
            output_labels.to(device)

            precision_metric.update(output_labels, validation_labels)
            recall_metric.update(output_labels, validation_labels)
            f1score_metric.update(output_labels, validation_labels)

            del validation_inputs, validation_labels, output_labels

            progress_bar.update(1)

    precision = precision_metric.compute().item()
    recall = recall_metric.compute().item()
    f1score = f1score_metric.compute().item()

    average_validation_loss = running_validation_loss / len(validation_dataloader)

    # Log the running loss averaged per batch
    # for both training and validation
    tensorboard_writer.add_scalars('Training vs. Validation Loss',
                                  { 'Training': avg_loss,
                                    'Validation': average_validation_loss},
                                    epoch)

    tensorboard_writer.add_scalars('Validation Metrics',
                                  { 'Precision': precision,
                                    'Recall': recall,
                                    'F1 Score': f1score},
                                    epoch)
    tensorboard_writer.flush()

    tensorboard_writer.add_scalars('Learning Rate',
                                  {'lr': optimizer.state_dict()['param_groups'][0]['lr']},
                                    epoch)
    tensorboard_writer.flush()

    # Track best performance, and save the model's state
    if((average_validation_loss < best_validation_loss) or (best_validation_loss == -1)):
        best_validation_loss = average_validation_loss
        torch.save(model.state_dict(), f'model_{timestamp}_best')

    torch.save(model.state_dict(), f'model_{timestamp}_{epoch}')

## Validação do modelo

In [ ]:
import math
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from torcheval.metrics import MulticlassConfusionMatrix

def test_model(model, device, test_dataset, batch_size=1, run_tests=True, time_test_iterations=1000, show_confusion_matrix=True, test_time=True):

    if(run_tests and show_confusion_matrix):
        confusion_matrix_metric = MulticlassConfusionMatrix(num_classes=len(test_dataset.classes))

    if(run_tests):
        precision_metric = MulticlassPrecision(average='weighted', num_classes=len(test_dataset.classes))
        recall_metric = MulticlassRecall(average='weighted', num_classes=len(test_dataset.classes))
        f1score_metric = MulticlassF1Score(average='weighted', num_classes=len(test_dataset.classes))

    model.eval()
    model.to(device)

    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

    if(run_tests):
        progress_bar = tqdm(total=len(test_dataloader))
        for test_data in test_dataloader:
            test_inputs, test_labels = test_data
            test_inputs = test_inputs.to(device)
            test_labels = test_labels.to(device)
            test_outputs = model(test_inputs)
            output_labels = []

            for j in range(0, test_outputs.size()[0]):
                probabilities = torch.nn.functional.softmax(test_outputs[j], dim=0)
                prob, det_class = torch.topk(probabilities, 1)
                # If the detected class is outside of bounds, give a wrong result within bounds
                if(det_class >= len(test_dataset.classes)):
                    det_class = test_labels[j].item()-1
                    if(det_class < 0):
                        det_class = test_labels[j].item()+1
                output_labels.append(det_class)

            output_labels = torch.as_tensor(output_labels)
            output_labels.to(device)

            confusion_matrix_metric.update(output_labels, test_labels)
            precision_metric.update(output_labels, test_labels)
            recall_metric.update(output_labels, test_labels)
            f1score_metric.update(output_labels, test_labels)

            del test_inputs, test_labels

            progress_bar.update(1)

        precision = precision_metric.compute().item()
        recall = recall_metric.compute().item()
        f1score = f1score_metric.compute().item()

        if(show_confusion_matrix):
            confusion_matrix_dataframe = pd.DataFrame(confusion_matrix_metric.normalized(),
                                                      index = test_dataset.classes,
                                                      columns = test_dataset.classes)
            plt.figure(figsize = (16,12))
            ax = sn.heatmap(confusion_matrix_dataframe, annot=True, cmap = 'YlOrBr')
            ax.xaxis.tick_top()
            ax.set_xticks(range(1, len(test_dataset.classes)+1), test_dataset.classes, rotation=270, ha='right');

    else:
        precision = 0.0
        recall = 0.0
        f1score = 0.0


    # Como o Colab possui um tempo de execução muito variável,
    # o tempo pode ser definido como constante, usando um dos valores testados.
    average_time = 0.00565 # Resnet 50, 160x160

    if(test_time):
        model_input, _ = next(iter(test_dataloader))
        model_input = model_input.to(device)

        total_time = 0
        for i in range(time_test_iterations):
            t0 = datetime.now()
            outputs = model(model_input)
            t1 = datetime.now()
            total_time += (t1-t0).total_seconds()

        del model_input

        average_time = total_time/time_test_iterations

    final_score = 10*f1score*math.pow(1-average_time, 2)

    return final_score, {"average_time": average_time, "f1score": f1score, "precision": precision, "recall": recall}

In [ ]:
best_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50')
best_model.load_state_dict(torch.load(f'model_{timestamp}_best'))

test_results = test_model(model=best_model,
                          device=device,
                          test_dataset=validation_dataset,
                          batch_size=1,
                          run_tests=True,
                          time_test_iterations=1000,
                          show_confusion_matrix=True)

print(f"Test data for model:")
print(f"  '-> Final Score: {test_results[0]}")
print(f"      |-> Average Time: {test_results[1]['average_time']}")
print(f"      '-> F1 Score:     {test_results[1]['f1score']}")
print(f"          |-> Precision: {test_results[1]['precision']}")
print(f"          '-> Recall:    {test_results[1]['recall']}")
